In [ ]:
## Download laz tiles ##

In [5]:
from API_utils import *
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [8]:
t0, laz = generate_laz_download_info(productcode, site, date)

results = []
for f in laz:
    results.append(delayed(download_from_NEON_API)(f, data_path))
    
with ProgressBar():
    computed = compute(*results)

[########################################] | 100% Completed |  2min  2.0s


In [4]:
site = 'TEAK'
productcode = 'DP1.30003.001'
data_path = '/home/jovyan/tmp'
date = '2018-06'
os.makedirs(data_path, exist_ok=True)

## Use LAStile ##

In [9]:
!cd LAStools/bin

In [18]:
!lastile -full_bb -i /home/jovyan/tmp/*.las -o /home/jovyan/tmp/NEON_D17_TEAK_DP1_classified_point_cloud_colorized_mosaic.laz 

/bin/bash: lastile: command not found
